<a href="https://colab.research.google.com/github/ZhengHuocheng/ZhengHuocheng/blob/main/bert_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import json
with open('train-v2.0.json', 'r') as f:
    train_data = json.load(f)
train_data = [item for topic in train_data['data'] for item in topic['paragraphs'] ]

In [9]:
split_index = int(len(train_data)*0.7)

In [10]:
train = train_data[0:split_index]
test = train_data[split_index:]
print(len(train))
print(len(test))

13324
5711


In [11]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [12]:
model_type="bert"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [7]:
!pip install wandb 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": False,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

In [14]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

In [15]:
model.train_model(train)

add example index and unique id: 100%|██████████| 89556/89556 [00:00<00:00, 859672.01it/s]


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


...

Running Epoch 1 of 5:   0%|          | 0/707 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/707 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/707 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/707 [00:00<?, ?it/s]

(3535, 0.7232958554715719)

In [16]:
result, texts = model.eval_model(test)

add example index and unique id: 100%|██████████| 40763/40763 [00:00<00:00, 649618.39it/s]


Running Evaluation:   0%|          | 0/644 [00:00<?, ?it/s]

In [17]:
result

{'correct': 21872,
 'similar': 16843,
 'incorrect': 2048,
 'eval_loss': -7.565878542313665}

In [18]:
to_predict = [
    {
        "context": "Jijun is the  most handsome man.",
        "qas": [
            {
                "question": "How to describe Jijun?",
                "id": "0",
            }
        ],
    }
]

In [36]:
answers, probabilities = model.predict(to_predict)

print(answers)
print(probabilities)     

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 3061.54it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['the most handsome man', 'the most handsome', '=Jijun is the most handsome man']}]
[{'id': '0', 'probability': [0.538209111319153, 0.2995584830357158, 0.1622321635090437]}]


In [29]:
model.save_model("model")

In [22]:
!pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
import streamlit as st

# Streamlit应用程序设置
st.title("问答系统")



DeltaGenerator()

In [42]:
!streamlit hello




  Welcome to Streamlit. Check out our demo in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.143.161.86:8501

  Ready to create your own Python apps super quickly?
  Head over to https://docs.streamlit.io

  May you create awesome apps!


  Stopping...
